<a href="https://colab.research.google.com/github/AkshathaBolla/ANN/blob/main/SmartGrid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Module I: Practice Exercise - 'Smart-Grid AI' (Big Arch Spec)

### **Context: The National Energy Grid**
You are the Lead Engineer for the National Grid. You have sensor readings from 100 power stations.
Your sensors measure:
1.  **Temperature (C)**
2.  **Wind Speed (km/h)**
3.  **Current Load (MW)**

You have **two separate goals**:
* **Goal A (Stability):** Predict if the grid is **'Unstable' (1)** or **'Stable' (0)**. (Classification)
* **Goal B (Demand):** Predict the **Next Hour Demand (MW)**. (Regression)

---
**INSTRUCTIONS:**
This exercise uses **Deeper and Wider Architectures**.
Pay attention to the layer dimensions (Input -> Hidden -> Output) to ensure the shapes match.

In [38]:
# CELL 1: DATA GENERATION (Run this first)
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

torch.manual_seed(500)

# 100 Power Stations
N = 100
# Features: Temp (-10 to 40), Wind (0-100), Load (1000-5000)
X = torch.rand(N, 3) * torch.tensor([50, 100, 4000]) + torch.tensor([-10, 0, 1000])

# Target A: Instability (Classification)
# Unstable if (Temp > 35 AND Load > 4500) OR Wind > 90
stability_score = (X[:, 0] - 35) + (X[:, 2] - 4500)/100 + (X[:, 1] - 90) + torch.randn(N)*5
y_stable = (stability_score > 0).float().view(-1, 1)

# Target B: Demand (Regression)
# Demand driven by Temp (AC/Heating) and current Load
y_demand = (X[:, 2] * 1.1) + (X[:, 0].abs() * 20) + torch.randn(N) * 50
y_demand = y_demand.view(-1, 1).float()

print(f"Data Ready. X: {X.shape}, y_stable: {y_stable.shape}, y_demand: {y_demand.shape}")

Data Ready. X: torch.Size([100, 3]), y_stable: torch.Size([100, 1]), y_demand: torch.Size([100, 1])


## **Level 1: The Blackout Predictor (Deep Classification)**
**Your Task:** Detect Instability.

**Blueprint (The 'Funnel' Architecture):**
1.  **Input:** 3 Features
2.  **Layer 1:** 64 Neurons, `Tanh`
3.  **Layer 2:** 32 Neurons, `Tanh`
4.  **Layer 3:** 16 Neurons, `Tanh`
5.  **Output:** 1 Neuron, `Sigmoid`

**Training Specs:**
* Loss: `BCELoss`
* Optimizer: `SGD` (lr=0.01)
* Epochs: 200

In [39]:
# LEVEL 1: WRITE YOUR CODE HERE

# 1. Define 'GridModel'
class GridModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.linear1=nn.Linear(3,64)
    self.layer1=nn.Tanh()
    self.linear2=nn.Linear(64,32)
    self.layer2=nn.Tanh()
    self.linear3=nn.Linear(32,16)
    self.layer3=nn.Tanh()
    self.linear4=nn.Linear(16,1)
    self.layer4=nn.Sigmoid()


    # TODO: Implement the 4-layer funnel
  def forward(self,x):
    x=self.linear1(x)
    x=self.layer1(x)
    x=self.linear2(x)
    x=self.layer2(x)
    x=self.linear3(x)
    x=self.layer3(x)
    x=self.linear4(x)
    x=self.layer4(x)
    return x;

model_grid = GridModel()
criterion=nn.BCELoss();
optimizer=optim.SGD(model_grid.parameters(),lr=0.01)

# 2. Optimizer & Loss
for epoch in range(200):
  optimizer.zero_grad()
  pred=model_grid(X);
  loss=criterion(pred,y_stable)
  loss.backward()
  optimizer.step()



# 3. Training Loop

# 4. Accuracy Check
with torch.no_grad():
  outputs=model_grid(X);
  pred=(outputs>=0.5).float()
  acc= (pred==y_stable).float().mean()
  print(f"Accuracy:{acc.item()}")

Accuracy:0.9900000095367432


In [40]:
# TEST LEVEL 1
try:
    # Check Layer Count
    layers = [m for m in model_grid.modules() if isinstance(m, nn.Linear)]
    assert len(layers) == 4, f"Blueprint requires 4 Linear layers. Found {len(layers)}"

    # Check Funnel Shapes
    assert layers[0].out_features == 64
    assert layers[1].out_features == 32
    assert layers[2].out_features == 16

    print("✅ Level 1 Passed: The Funnel Architecture is correct.")
except Exception as e:
    print(f"❌ Level 1 Fail: {e}")

✅ Level 1 Passed: The Funnel Architecture is correct.


## **Level 2: Demand Forecasting (Wide Regression)**
**Your Task:** Predict Demand (MW).

**Step 2.1: Pipeline**
* Split Train(20) / Test(80). `DataLoader` (Batch=10).

**Step 2.2: The 'Fat' Model (Overfitting Risk)**
* `Linear(3, 256) -> ReLU`
* `Linear(256, 256) -> ReLU`
* `Linear(256, 1)`

**Step 2.3: Diagnosis**
* Train for 300 epochs. Confirm Overfitting.

**Step 2.4: The Fix (Dropout)**
* Rebuild with `nn.Dropout(0.5)` after the first two ReLUs.
* Retrain.

In [41]:
# LEVEL 2: WRITE YOUR CODE HERE

# 1. Data Pipeline
X_train,X_test=X[:20],X[20:]
y_train,y_test=y_stable[:20],y_stable[20:]

train_dataset=TensorDataset(X_train,y_train)
test_dataset=TensorDataset(X_test,y_test)

train_dataloader=DataLoader(train_dataset,batch_size=10,shuffle=True)
test_dataloader=DataLoader(test_dataset,batch_size=10)

# 2. Define 'model_fat' (256 width)
class ModelFat(nn.Module):
  def __init__(self):
    super().__init__()
    self.layer1=nn.Linear(3,256)
    self.layer2=nn.ReLU()
    self.layer3=nn.Linear(256,256)
    self.layer4=nn.ReLU()
    self.layer5=nn.Linear(256,1)

  def forward(self,x):
    x=self.layer1(x)
    x=self.layer2(x)
    x=self.layer3(x)
    x=self.layer4(x)
    x=self.layer5(x)
    return x
model_fat=ModelFat()
# 3. Train Loop
optimizer=optim.SGD(model_fat.parameters(),lr=0.01)
criterion=nn.MSELoss()
for epoch in range(300):
  model_fat.train()
  for inputs,target in train_dataloader:
    optimizer.zero_grad()
    pred=model_fat(inputs)
    loss=criterion(pred,target)
    loss.backward()
    optimizer.step()

 # 4. Define 'model_smart' (With Dropout 0.5)
class ModelSmart(nn.Module):
  def __init__(self):
    super().__init__()
    self.layer1=nn.Linear(3,256)
    self.layer2=nn.ReLU()
    self.droput1=nn.Dropout(0.5)
    self.layer3=nn.Linear(256,256)
    self.layer4=nn.ReLU()
    self.dropout2=nn.Dropout(0.5)
    self.layer5=nn.Linear(256,1)

  def forward(self,x):
    x=self.layer1(x)
    x=self.layer2(x)
    x=self.droput1(x)
    x=self.layer3(x)
    x=self.layer4(x)
    x=self.dropout2(x)
    x=self.layer5(x)
    return x
model_smart=ModelSmart()
optimizer=optim.SGD(model_smart.parameters(),lr=0.01)
criterion=nn.MSELoss()
for epoch in range(300):
  model_smart.train()
  for inputs,target in train_dataloader:
    optimizer.zero_grad()
    target=target.view(-1,1)
    pred=model_smart(inputs)
    loss=criterion(pred,target)
    loss.backward()
    optimizer.step()

In [42]:
# TEST LEVEL 2
try:
    # Check Width
    l1 = list(model_smart.modules())[1]
    if isinstance(l1, nn.Sequential): l1 = l1[0]
    assert l1.out_features == 256, f"Hidden layer must be 256 wide. Found {l1.out_features}"

    # Check Dropout
    drop = [m for m in model_smart.modules() if isinstance(m, nn.Dropout)]
    assert len(drop) >= 2, "Missing dropout layers"
    assert drop[0].p == 0.5, "Dropout rate must be 0.5"

    print("✅ Level 2 Passed: Wide Architecture built correctly.")
except Exception as e:
    print(f"❌ Level 2 Fail: {e}")

✅ Level 2 Passed: Wide Architecture built correctly.


## **Level 3: The Architect (Manual MAE)**

**Part 3.1: Manual MAE (Mean Absolute Error)**
Implement `my_custom_mae`.
$$ Loss = Mean( |y_{pred} - y_{true}| ) $$
*Hint: Use `torch.abs()`.*

**Part 3.2: The Tournament**
Compare these 3 configs on **Demand Prediction**:

1.  **"Standard"**: `Linear(3, 64) -> ReLU -> Linear(64, 1)` (Loss: `MSELoss`)
2.  **"Leaky"**: `Linear(3, 64) -> LeakyReLU(0.1) -> Linear(64, 1)` (Loss: `L1Loss`)
3.  **"Mobile"**: `Linear(3, 64) -> Hardswish -> Linear(64, 1)` (Loss: **YOUR** `my_custom_mae`)

**Note:** `Hardswish` is a memory-efficient activation used in MobileNetV3.

In [43]:
# LEVEL 3.1: MANUAL MAE
def my_custom_mae(pred, target):
    # TODO: Implement Mean Absolute Error
    abs_error=torch.abs(pred-target)
    return torch.mean(abs_error)

# LEVEL 3.2: TOURNAMENT
experiments = [
    {"name": "Standard", "act": nn.ReLU(), "loss": nn.MSELoss()}, # ReLU, MSE
    {"name": "Leaky",    "act": nn.LeakyReLU(), "loss": nn.L1Loss()}, # LeakyReLU(0.1), L1
    {"name": "Mobile",   "act": nn.Hardswish(), "loss": my_custom_mae}  # Hardswish, Custom MAE
]

print(f"{'Name':<10} | {'Test Loss':<10}")
print("-"*25)

# Loop...

Name       | Test Loss 
-------------------------


In [44]:
# TEST LEVEL 3
try:
    # Test Math
    p = torch.tensor([-2.0]); t = torch.tensor([2.0])
    assert my_custom_mae(p, t).item() == 4.0, "Math Fail: |-2 - 2| should be 4"

    # Check Configs
    assert isinstance(experiments[2]['act'], nn.Hardswish), "Exp 3 Act must be Hardswish"
    assert not isinstance(experiments[2]['loss'], type), "Exp 3 Loss must be custom function"

    print("✅ Level 3 Passed: Manual MAE and Configs valid.")
except Exception as e:
    print(f"❌ Level 3 Fail: {e}")

✅ Level 3 Passed: Manual MAE and Configs valid.


## **Level 4: The Mechanic (Manual Optimization)**
**Your Task:** The `optim` library is deleted. Train a simple Linear Regression model manually.

1.  Create a single layer `nn.Linear(1, 1)`.
2.  Write a loop that updates weights using: `w = w - lr * gradient`.
3.  **Important:** You must use `with torch.no_grad():` for the update step.

In [45]:
# LEVEL 4: THE PURGE (Run this to delete optimizers)
import torch.optim as optim
import gc
for var in list(locals().keys()):
    if 'opt' in var or 'optimizer' in var:
        del locals()[var]
gc.collect()
print("Optimizers deleted. You are on your own.")

Optimizers deleted. You are on your own.


In [50]:
# LEVEL 4: WRITE YOUR CODE HERE

# 1. Setup Data
X_simple = torch.tensor([[1.0], [2.0], [3.0], [4.0]])
y_simple = torch.tensor([[3.0], [5.0], [7.0], [9.0]]) # Target: y = 2x + 1

# 2. Define Model
model_manual = nn.Linear(1,1)
lr=0.01
criterion=nn.MSELoss()
for epoch in range(3000):
  preds=model_manual(X_simple)
  loss=criterion(preds,y_simple)
  loss.backward()
  with torch.no_grad():
        model_manual.weight-=lr*model_manual.weight.grad

# 3. Manual Loop

    # Forward, Loss, Backward
    # Update: param -= lr * param.grad
    # Zero grad
  model_manual.weight.grad.zero_()


In [51]:
# TEST LEVEL 4
try:
    w = model_manual.weight.item()
    b = model_manual.bias.item()
    assert abs(w - 2.0) < 0.2, f"Convergence Fail: Weight is {w:.2f}, expected ~2.0"
    print("✅ Level 4 Passed: Manual Optimization successful!")
except Exception as e:
    print(f"❌ Level 4 Fail: {e}")

✅ Level 4 Passed: Manual Optimization successful!
